In [1]:
# ==================================================================================================
# Yufei Li
# Princeton University
# yl5385@princeton.edu

# Feburay 2023

# Note:In this demo, the neural network is synthesized using the TensorFlow (verion: 2.11.0) framework. 
# Please install TensorFlow according to the official guidance, then import TensorFlow and other dependent modules.
# ==================================================================================================

!pip install pandas numpy matplotlib
!pip install tensorflow
!pip install optuna

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


ERROR: Exception:
Traceback (most recent call last):
  File "D:\hard\anaconda\lib\site-packages\pip\_internal\cli\base_command.py", line 173, in _main
    status = self.run(options, args)
  File "D:\hard\anaconda\lib\site-packages\pip\_internal\cli\req_command.py", line 203, in wrapper
    return func(self, options, args)
  File "D:\hard\anaconda\lib\site-packages\pip\_internal\commands\install.py", line 315, in run
    requirement_set = resolver.resolve(
  File "D:\hard\anaconda\lib\site-packages\pip\_internal\resolution\resolvelib\resolver.py", line 94, in resolve
    result = self._result = resolver.resolve(
  File "D:\hard\anaconda\lib\site-packages\pip\_vendor\resolvelib\resolvers.py", line 472, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "D:\hard\anaconda\lib\site-packages\pip\_vendor\resolvelib\resolvers.py", line 341, in resolve
    self._add_to_criteria(self.state.criteria, r, parent=None)
  File "D:\hard\anaconda\lib\site-packages\pip

In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import copy
import csv
import math
import cmath
import time

import optuna

inFilename = "Input_Yo20op272_ana.csv"
outFilename = "Output_Yo20op272_ana.csv"

Input = pd.read_csv(inFilename,header=None)
Output = pd.read_csv(outFilename,header=None)

print(Input.dtypes)
print(Output.dtypes)

inputs = []
outputs = []

inputs = np.array(Input)
outputs = np.array(Output)
print(inputs)
print(outputs)

0    float64
1    float64
2    float64
3    float64
dtype: object
0    float64
1    float64
2    float64
3    float64
4    float64
5    float64
6    float64
7    float64
dtype: object
[[  1.           0.9         -0.8         -0.4       ]
 [  1.31578947   0.9         -0.8         -0.4       ]
 [  1.75438596   0.9         -0.8         -0.4       ]
 ...
 [116.66666667   1.1          1.           0.6       ]
 [150.           1.1          1.           0.6       ]
 [200.           1.1          1.           0.6       ]]
[[ 5.31458012e-05  2.29059616e-03 -3.24377888e-02 ... -3.09055016e-07
   6.48620564e-02 -3.33011736e-04]
 [ 9.19777914e-05  3.01278965e-03 -3.29274705e-02 ... -7.03498158e-07
   6.58173966e-02 -7.58000403e-04]
 [ 1.63406069e-04  4.01422579e-03 -3.37875330e-02 ... -1.66520633e-06
   6.74621041e-02 -1.79157180e-03]
 ...
 [ 8.83554162e-02  2.91640801e-02  8.41353765e-03 ... -6.11382404e-03
   8.65013151e-02  4.29239326e-02]
 [ 9.28972074e-02  2.24226852e-02  1.01270630e-02 ... -

In [3]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation

num_inputs = len(inputs)
print("Total Number of Dataset is:",num_inputs)
randomize = np.arange(num_inputs)
print(randomize)

inputs_origin = copy.deepcopy(inputs[randomize])
outputs_origin = copy.deepcopy(outputs[randomize])
print(inputs_origin)

random.Random(5).shuffle(randomize)
print(randomize)
# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs_real = copy.deepcopy(inputs_origin[randomize])
outputs_real = copy.deepcopy(outputs_origin[randomize])
print(inputs_real)
print(outputs_real)

# Split the recordings (group of samples) into two sets: training and testing
TRAIN_SPLIT = int(0.7 * num_inputs)
inputs_train, inputs_test = np.split(inputs_real, [TRAIN_SPLIT])
outputs_train, outputs_test = np.split(outputs_real, [TRAIN_SPLIT])

num_inputs_train = len(inputs_train)
print("Total Number of training Dataset is:",num_inputs_train)
print("Dataset randomization and separation complete!")

Total Number of Dataset is: 5440
[   0    1    2 ... 5437 5438 5439]
[[  1.           0.9         -0.8         -0.4       ]
 [  1.31578947   0.9         -0.8         -0.4       ]
 [  1.75438596   0.9         -0.8         -0.4       ]
 ...
 [116.66666667   1.1          1.           0.6       ]
 [150.           1.1          1.           0.6       ]
 [200.           1.1          1.           0.6       ]]
[ 689 2176 5047 ... 2937 2092 5102]
[[ 12.5          0.9          0.           0.2       ]
 [ 87.5          1.          -0.2          0.6       ]
 [  6.97674419   1.1          0.6          0.4       ]
 ...
 [116.66666667   1.           0.6         -0.2       ]
 [ 28.57142857   1.          -0.2         -0.2       ]
 [  1.75438596   1.1          0.6          1.        ]]
[[ 7.70282837e-03  2.64698365e-02  3.92927899e-03 ... -5.16123709e-04
  -1.32699782e-02  2.40342759e-02]
 [ 8.08593968e-02  3.70515446e-02  5.82464384e-03 ... -7.37553062e-03
   7.68197520e-02  4.49022591e-02]
 [ 2.52653115

In [4]:
EPOCHS = 500

def objective(trial):
    #Clear clutter from previous tf.keras session graphs.
    #tf.keras.backend.clear_session()
    
    #Define normalization layer
    Normlayer1=tf.keras.layers.Normalization()
    Normlayer1.adapt(inputs_real)
    
    #Model construction
    #Optimize the numbers of layers and their units.
    n_layers = trial.suggest_int("n_layers", 2, 6)
    model = tf.keras.Sequential()    
    model.add(Normlayer1)
    for i in range(n_layers):
        neurons_hidden = trial.suggest_int("n_units_l{}".format(i), 32, 128, log=True)
        model.add(tf.keras.layers.Dense(neurons_hidden, activation="sigmoid"))
    model.add(tf.keras.layers.Dense(8))

    #Defining learning rate schedule
    LR_INIT = trial.suggest_float("lr_init", 1e-3, 5e-1, log=True)
    DECAY_EPOCHS = trial.suggest_int("decay_epochs", 10, 200, log=True)
    DECAY_RATE = trial.suggest_float("decay_rate", 0.4, 0.9, log=True)
    BATCH_SIZE = trial.suggest_categorical("batch_size", [16, 32, 64, 128, 256, 512, 1024])

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    LR_INIT,
    decay_steps=math.ceil(num_inputs_train/BATCH_SIZE)*DECAY_EPOCHS,
    decay_rate=DECAY_RATE,
    staircase=True)
    
    #Compile model
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])
    
    history = model.fit(inputs_train, outputs_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(inputs_test, outputs_test), verbose=0)

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(inputs_test, outputs_test, verbose=0)
    return score[1]
print("Optimization object construction complete!")

Optimization object construction complete!


In [5]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100)
    
    print("Number of finished trials: ", len(study.trials))       
    
    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-06-19 17:07:55,259] A new study created in memory with name: no-name-8d50cc9f-ccfc-4f3a-8522-6df0a405fbb2
[I 2023-06-19 17:08:18,589] Trial 0 finished with value: 0.000657229742500931 and parameters: {'n_layers': 4, 'n_units_l0': 47, 'n_units_l1': 83, 'n_units_l2': 59, 'n_units_l3': 58, 'lr_init': 0.26742894488188607, 'decay_epochs': 34, 'decay_rate': 0.7232087989932243, 'batch_size': 128}. Best is trial 0 with value: 0.000657229742500931.
[I 2023-06-19 17:08:40,531] Trial 1 finished with value: 3.9701561036054045e-05 and parameters: {'n_layers': 2, 'n_units_l0': 78, 'n_units_l1': 63, 'lr_init': 0.0018536847107245322, 'decay_epochs': 24, 'decay_rate': 0.80542525648899, 'batch_size': 128}. Best is trial 1 with value: 3.9701561036054045e-05.
[I 2023-06-19 17:10:20,763] Trial 2 finished with value: 2.647438712699568e-08 and parameters: {'n_layers': 4, 'n_units_l0': 122, 'n_units_l1': 102, 'n_units_l2': 45, 'n_units_l3': 48, 'lr_init': 0.008607561000278378, 'decay_epochs': 38, 'dec

Number of finished trials:  100
Best trial:
  Value:  9.062891770383885e-09
  Params: 
    n_layers: 3
    n_units_l0: 84
    n_units_l1: 111
    n_units_l2: 32
    lr_init: 0.011703102615006086
    decay_epochs: 42
    decay_rate: 0.6865596993090048
    batch_size: 16


In [17]:
optuna.visualization.plot_optimization_history(study)#Plotting the optimization history of the study.

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [18]:
optuna.visualization.plot_intermediate_values(study)#Visualizing the Learning Curves of the Trials

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [19]:
optuna.visualization.plot_parallel_coordinate(study)#Visualizing High-dimensional Parameter Relationships

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [20]:
optuna.visualization.plot_parallel_coordinate(study, params=['lr_init', 'n_layers', 'n_units_l0', 'n_units_l1', 'n_units_l2', 'n_units_l3', 'decay_epochs', 'decay_rate'])#Selecting Parameters to Visualize

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [11]:
optuna.visualization.plot_contour(study)#Visualizing Parameter Relationships

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [12]:
optuna.visualization.plot_contour(study, params=['n_units_l0', 'n_units_l1'])#Selecting Parameters to Visualize

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [13]:
optuna.visualization.plot_slice(study)#Plotting the accuracies for each hyperparameter for each trial. Visualizing Individual Parameters

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [14]:
optuna.visualization.plot_slice(study, params=['n_units_l0', 'n_units_l1'])#Selecting Parameters to Visualize

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [15]:
optuna.visualization.plot_param_importances(study)#Visualizing Parameter Importances

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [16]:
optuna.visualization.plot_edf(study)

ImportError: Tried to import '_plotly_utils' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named '_plotly_utils'.

In [21]:
print(study)

In [25]:
import pickle
def save_variable(v,filename):
    f=open(filename,'wb')
    pickle.dump(v,f)
    f.close()
    return filename
def load_variavle(filename):
    f=open(filename,'rb')
    r=pickle.load(f)
    f.close()
    return r
# a = save_variable(study, 'study.txt')
study1 = load_variavle('study.txt')

In [30]:
print(study)
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Value:  9.062891770383885e-09
  Params: 
    n_layers: 3
    n_units_l0: 84
    n_units_l1: 111
    n_units_l2: 32
    lr_init: 0.011703102615006086
    decay_epochs: 42
    decay_rate: 0.6865596993090048
    batch_size: 16


In [29]:
print(study1)
trial1 = study1.best_trial
print("  Value: ", trial1.value)
print("  Params: ")
for key, value in trial1.params.items():
    print("    {}: {}".format(key, value))

  Value:  9.062891770383885e-09
  Params: 
    n_layers: 3
    n_units_l0: 84
    n_units_l1: 111
    n_units_l2: 32
    lr_init: 0.011703102615006086
    decay_epochs: 42
    decay_rate: 0.6865596993090048
    batch_size: 16
